## Truncamento das imagens e importação das bibliotecas

In [ ]:
#TRUNCAMENTO das imagens e importação das bibliotecas
import numpy as np
from osgeo import gdal #biblioteca de manipulaçao das imagens GIS
import os

def saveRaster(dataset, datasetPath, cols, rows, projection, bands):
    driver = gdal.GetDriverByName('GTiff')
    
    outRaster = driver.Create(datasetPath, cols, rows, bands, gdal.GDT_Float32)
    outRaster.SetGeoTransform(geotransform)
    outRaster.SetProjection(projection)
    for b in range(bands):
        outband = outRaster.GetRasterBand(b + 1)
        outband.WriteArray(dataset[b])
        outband.SetNoDataValue(-32768)
    outRaster = None

PathIn = r"/home/nico/dev/cicatrizes_certo/bandas/" #Pasta com as 10 bandas recortadas
PathOut = r"/home/nico/dev/cicatrizes_certo/composicoes/" #Pasta para salvar as 10 bandas truncadas


imageList = [img for img in os.listdir(PathIn) if img[-4:]=='.tif']
print(f"Número total de imagens: {len(imageList)}", end= "\n")

images_Normalizadas = []
for image in imageList: 
    print(f"Imagen: {image}")
    Im = gdal.Open(PathIn + image)
    bands = Im.RasterCount
    bandas = [Im.GetRasterBand(b).ReadAsArray().astype(float) for b in range(1, bands + 1)]
    for i, band in enumerate(bandas):
        print(f"Banda {i + 1} valor máximo antes: {np.max(band)}")
        bandas[i] = np.where((band > 10000), 10000, band)
        print(f"Banda {i + 1} valor máximo depois: {np.max(bandas[i])}")
    images_Normalizadas.append(bandas)

imProva = gdal.Open(PathIn+imageList[0]) 
geotransform = imProva.GetGeoTransform()
originX, pixelWidth, empty, finalY, empty2, pixelHeight = geotransform
cols =  imProva.RasterXSize
rows =  imProva.RasterYSize
projection = imProva.GetProjection()

PathName = []
for image in imageList:
    Path = PathOut + image[:-4]+'_truncada'+'.tif'
    PathName.append(Path)    
print(f"Número total imagens: {len(PathName)}", end= "\n")

for i in range(len(PathName)):
    cols = images_Normalizadas[i][0].shape[1]
    rows = images_Normalizadas[i][0].shape[0]
    saveRaster(images_Normalizadas[i], PathName[i], cols, rows, projection, len(images_Normalizadas[i]))

## Reescalar imagens , para deixar as imagens de mesma dimensao e valor de bandas

In [ ]:
#re_escalar imagens , para deixar as imagens de mesma dimensao 
import os
from osgeo import gdal, gdal_array
import numpy as np

def saveRaster(datasets, datasetPath, cols, rows, projection, bands):
    rasterSet = gdal.GetDriverByName('GTiff').Create(datasetPath, cols, rows, bands, gdal.GDT_Float32)
    rasterSet.SetProjection(projection)
    rasterSet.SetGeoTransform(geotransform)
    for i in range(bands):
        rasterSet.GetRasterBand(i+1).WriteArray(datasets[i])
        rasterSet.GetRasterBand(i+1).SetNoDataValue(-32768)
    rasterSet = None

def Normalizar(M):  
    Max = np.max(M)
    masked = np.ma.masked_equal(M, -32768, copy=False)
    Min = np.min(masked)
    print(f"\nVERDADEiRO MÍNIMO: {Min} \n")    
    if Max != Min:
        R = np.where(
        (M)== -32768,
        -32768,
        (M-Min)/(Max-Min)
        )
        return R
    else:
        print(f"Nao pode normalizar")

PathImageFiles = r"/home/nico/dev/cicatrizes_certo/shapeFile/"
PathOuputFiles = r"/home/nico/dev/cicatrizes_certo/composicoes_out/"

bandsList = [band for band in os.listdir(PathImageFiles) if band[-4:]=='.tif']

imProva= gdal.Open(PathImageFiles + bandsList[0])
geotransform = imProva.GetGeoTransform()
projection = imProva.GetProjection()

for band in bandsList:
    raster = gdal.Open(PathImageFiles + band)
    bands = raster.RasterCount
    img = [raster.GetRasterBand(i).ReadAsArray() for i in range(1, bands+1)]
    img_normalized = []
    for i, im in enumerate(img):
        print(f"\nBefore Banda {i+1} --> Maximo: {np.max(im)}  Minimo: {np.min(im)}")
        im = Normalizar(im)
        print(f"\nAfter Banda {i+1} --> Maximo: {np.max(im)}  Minimo: {np.min(im)}")
        img_normalized.append(im)
    saveRaster(img_normalized, PathOuputFiles+band[:-4]+"_reescalado.tif", img_normalized[0].shape[1], img_normalized[0].shape[0],projection, len(img_normalized))



## Desenhar manualmente com pontos as cicatrizes nas imagens e salvar em arquivo txt as coordenadas da mesma

In [1]:
#desenhar as cicatrizes nas imagens e salvar em arquivo txt as coordenadas da mesma
import cv2 as cv
import numpy as np
from osgeo import ogr, gdal, osr

#Funcao para normalizar as imagens
def Normalizar(M):  
    Max = np.max(M)
    Min = np.min(M)
    if Max != Min:
        R=255*np.divide(M-Min,Max-Min, where = (Max-Min)!=0)
        RU = R.astype(np.uint8)
        return RU
    else:
        print("Nao pode normalizar ")


#Funcao de leitura da barra de amostra
def Seguir(x):   
    globals()['Y0']=cv.getTrackbarPos("Vertical","Desplazar")   
    globals()['X0']=cv.getTrackbarPos("Horizontal","Desplazar")


#Funcao para os eventos do mouse 
def Etiquetar(event,x,y,flags,param):   
   
    Y0 =globals()['Y0']
    X0 =globals()['X0']
    
        
    xg=x+X0
    yg=y+Y0
    xg2=x+X0
    yg2=y+Y0   
    globals()['sair'] = sair;    

    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(ImagenRoi_cicatrizes,(x,y),1,(0,0,255),-1)
        f.write('%s'%xg)
        f.write(',')
        f.write('%s' %yg) 
        f.write(',')
        f.write('%s'  %globals()['Classe'])
        f.write(' ')
        f.write('\n')
                     
    if event == cv.EVENT_LBUTTONUP:
        globals()['Cont']= Cont + 1
         
    
    if event == cv.EVENT_MOUSEMOVE: 
            globals()['xp']= x;
            globals()['yp']= y;
        
    Titulo = 'Classe: ' + str(Classe) + " "+ 'Amostra: ' + str(Cont) +'  '+ 'x: '+ str(xg) + ' ' + 'y: ' + str(yg)
    cv.setWindowTitle("Imagen_Principal", Titulo)



# ENTRADAS do programa
Path1 = "/home/nico/dev/cicatrizes_certo/composicoes_out/" #Pasta com as bandas truncadas
Path2 = "/home/nico/dev/cicatrizes_certo/indices/" #Pasta com os indices
PathOuput = "/home/nico/dev/cicatrizes_certo/desenhadas/" #Pasta com destino final

band04 = 'merge_final_23_LKL_1_2_truncada_reescalado.tif' #Nome da imagem com 10 bandas


ImR = gdal.Open(Path1 + band04)
ImB8A = gdal.Open(Path1 + band04)
ImB11 = gdal.Open(Path1 + band04)
ImG = gdal.Open(Path1 + band04)
ImB = gdal.Open(Path1 + band04)
ImB12 = gdal.Open(Path1 + band04)
#selecao das bandas para melhor visualizacao das cicatrizes na selecao das amostras
R= ImR.GetRasterBand(5).ReadAsArray().astype(float) #5 
B8A = ImB8A.GetRasterBand(4).ReadAsArray().astype(float) #4
B11 = ImB11.GetRasterBand(9).ReadAsArray().astype(float) #9
G= ImG.GetRasterBand(4).ReadAsArray().astype(float)
B= ImB.GetRasterBand(5).ReadAsArray().astype(float)
B12 = ImB12.GetRasterBand(6).ReadAsArray().astype(float)


cicatrizes = np.dstack([B11, B8A, R])/np.max(np.dstack([B11, B8A, R]))  #Classe: cicatrizes de fogo




########################################################  CLASSIFICACAO MANUAL ############################################################


Total_Clases = 1      #Numero de clases para fazer seleção de Amostras

Cont = 0
Classe = 1
sair=False;

ArquivoPontos = "pontos_other_23_LKL_1_2.txt" #Nome do arquivo de saída cuidar para nao sobrescrever

f = open (PathOuput + ArquivoPontos,'w')  #ARQUIVO DE SAIDA

cv.namedWindow('Imagen_Principal',cv.WINDOW_NORMAL) 


#Tamanho das imagens para mostrar
TamY=5490; 
TamX=5490; 
#Variaveis para mostrar as imagens
X0=0;
Y0=0;
xp=0;
yp=0;
cv.setMouseCallback('Imagen_Principal', Etiquetar)   
Offset = 5;
cicatrizes = Normalizar(cicatrizes)

while True:    
    X1 = globals()['X0'];  
    Y1 = globals()['Y0'];  
    ImagenRoi_cicatrizes = cicatrizes[Y1:TamY+Y1,X1:TamX+X1] 
    cv.imshow('Imagen_Principal',ImagenRoi_cicatrizes)
    k2 = cv.waitKey(1) & 0xFF
    if k2 == ord("q"):
        globals()['Classe'] = Classe + 1
        globals()['Cont'] =0
        
        if globals()['Total_Clases'] == Classe-1:
            globals()['sair'] = True

    k = cv.waitKey(1) & 0xFF
    if (k == 27) or sair==True:
        break

f.close()
cv.destroyAllWindows() 

## Build dataset, extrai as coordenadas das cicatrizes de uma imagem multibanda grande e salva em uma nova imagem de 5x5x10 (10 bandas)

In [4]:
#build dataset, extrai as coordenadas das cicatrizes e salva em uma nova imagem de 5x5x10 (10 bandas)
import csv
import numpy as np
import cv2 as cv
from osgeo import gdal
import os


class utils(object):
	def __init__(self,path_to_csv=None):
		if path_to_csv is None:
			self.path_to_csv = None
		else:
			self.path_to_csv = path_to_csv

	def set_path_to_csv(self,path_to_csv):
		self.path_to_csv = path_to_csv

	def write_pointers(self, pointers):
		with open(self.path_to_csv, 'a+', newline='') as file:
			writer = csv.writer(file, delimiter=',')
			writer.writerows(pointers)

	def read_pointers(self):
		pointers = []
		with open(self.path_to_csv, 'r') as file:
			reader = csv.reader(file, delimiter = ',')
			for row in reader:
				pointers.append(row)
		return pointers

	def set_ROI (self,img,punto,kernel):
		fi=punto[1]-kernel
		ff=punto[1]+kernel+1
		ci=punto[0]-kernel
		cf=punto[0]+kernel+1
		ROI = img[fi:ff,ci:cf] #building roi (rows(y) cols(x))
		return ROI

	def printProgressBar (self, iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
		"""
		Call in a loop to create terminal progress bar 
		@params:
			iteration   - Required  : current iteration (Int)
			total       - Required  : total iterations (Int)
			prefix      - Optional  : prefix string (Str)
			suffix      - Optional  : suffix string (Str)
			decimals    - Optional  : positive number of decimals in percent complete (Int)
			length      - Optional  : character length of bar (Int)
			fill        - Optional  : bar fill character (Str)
			printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
		"""
		percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
		filledLength = int(length * iteration // total)
		bar = fill * filledLength + '-' * (length - filledLength)
		print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
		# Print New Line on Complete
		if iteration == total: 
			print()
   
   


class handlingSI(object):
	def __init__(self, path_to_SAR_VV=None):
		self.Bands = []
		self.flag = None
		if(path_to_SAR_VV is None):
			self.path_to_SAR_VV = None
			self.raster = None
			self.SARimage = None
		else:
			self.path_to_SAR_VV = path_to_SAR_VV
			self.raster = gdal.Open(self.path_to_SAR_VV)
			if self.raster.RasterCount == 1:
				self.SARimage = self.raster.ReadAsArray()
				self.flag = 1
			else:
				in_band = self.raster.GetRasterBand(1)
				self.SARimage = in_band.ReadAsArray()
				self.flag = self.raster.RasterCount
				for i in range(self.raster.RasterCount):
					self.Bands.append(self.raster.GetRasterBand(i+1).ReadAsArray())


	def set_raster(self, raster): 
		self.raster = raster
		self.flag = self.raster.RasterCount
		if self.flag > 1:
			for i in range(self.raster.RasterCount):
				self.Bands.append(self.raster.GetRasterBand(i+1).ReadAsArray())

	def loadSI(self,path_to_SAR_VV): # metodo para carregar a imagem
		self.path_to_SAR_VV = path_to_SAR_VV
		self.raster = gdal.Open(self.path_to_SAR_VV)
		if self.raster.RasterCount == 1:
			self.SARimage = self.raster.ReadAsArray()
			self.flag= 1
		else:
			in_band = self.raster.GetRasterBand(1)
			self.SARimage = in_band.ReadAsArray()
			self.flag = self.raster.RasterCount
			for i in range(self.raster.RasterCount):
				self.Bands.append(self.raster.GetRasterBand(i+1).ReadAsArray()) 


	def saveSI(self,image,outFileName,NoDataValue = -32768): # metodo para salvar imagem
		driver = gdal.GetDriverByName("GTiff")
		#print("bandera: ",self.flag)
		#print("planos: ",len(self.Bands))
		if self.flag==1:
			outdata = driver.Create(outFileName,image.shape[1], image.shape[0], 1, gdal.GDT_Float32)
			outdata.SetGeoTransform(self.raster.GetGeoTransform())
			outdata.SetProjection(self.raster.GetProjection())
			outdata.GetRasterBand(1).WriteArray(image)
			outdata.GetRasterBand(1).SetNoDataValue(NoDataValue)
			outdata.FlushCache()
		else:			
			outdata = driver.Create(outFileName,image.shape[1], image.shape[0], self.flag, gdal.GDT_Float32)
			outdata.SetGeoTransform(self.raster.GetGeoTransform())
			outdata.SetProjection(self.raster.GetProjection())
			Bandas=cv.split(image)
			#print(f"Bandas: {len(Bandas)}; image: {image.shape}")
			#print("Bandas en saveSI", Bandas[0].shape)
			for i in range(self.flag):
				outdata.GetRasterBand(i+1).WriteArray(Bandas[i])
				#outdata.GetRasterBand(i+1).SetNoDataValue(NoDataValue)
			outdata.FlushCache()

	def saveSI_inference(self,image,outFileName,NoDataValue = -32768): 
		driver = gdal.GetDriverByName("GTiff")		
		outdata = driver.Create(outFileName,image.shape[1], image.shape[0], 1, gdal.GDT_Float32)
		outdata.SetGeoTransform(self.raster.GetGeoTransform())
		outdata.SetProjection(self.raster.GetProjection())
		outdata.GetRasterBand(1).WriteArray(image)
		outdata.GetRasterBand(1).SetNoDataValue(NoDataValue)
		outdata.FlushCache()
		


	def saveImage(self,image,outFileName):
		cv.imwrite(outFileName,image)


class buildDataset(object):
	def __init__(self, path_to_dataset, image, raster_from_gdal, kernel, points = None):
		self.path_to_dataset = path_to_dataset
		self.image = image
		self.raster = raster_from_gdal
		self.kernel = kernel
		if points is None:
			self.points = None
		else:
			self.points = points
			self.build()

	def get_points(self, path_to_csv):
		obutils = utils(path_to_csv)
		self.points = obutils.read_pointers()
		for i in range(len(self.points)):
			self.points[i][0] = int(self.points[i][0])
			self.points[i][1] = int(self.points[i][1])
		self.build()

	def build(self):
		obutils = utils()
		obhand = handlingSI()
		obhand.set_raster(self.raster)
		#print("Comprovar: ",len(obhand.Bands))
		for i in range(len(self.points)):
			foldername = self.path_to_dataset+"/"+str(self.points[i][2])
			if not os.path.exists(foldername):
				os.makedirs(foldername)
			saved_id=foldername+"/"+str(self.points[i][2])+"_"+str(self.points[i][0])+str(self.points[i][1])+".tif"
			refPt = [self.points[i][0],self.points[i][1]]
			delta_kernel=int(np.trunc(self.kernel/2))
			if self.raster.RasterCount == 1:
				roi=obutils.set_ROI(self.image,refPt,delta_kernel)
				row=roi.shape[0]
				col=roi.shape[1]
				if row == col and row == self.kernel:
					#print(roi.shape)
					obhand.saveSI(roi,saved_id,NoDataValue = -32768)
				else:
					print("nao salvo")
			else:
				multiband=cv.merge(obhand.Bands)
				roi=obutils.set_ROI(multiband,refPt,delta_kernel)
				row=roi.shape[0]
				col=roi.shape[1]
				#print("ROI_size: ",roi.shape)
				if row == col and row == self.kernel:
					#print(roi.shape)
					obhand.saveSI(roi,saved_id,NoDataValue = -32768)
				else:
					print("nao salvo")

# paths de entrada
path_to_SAR_VV = r"/home/nico/dev/cicatrizes_certo/composicoes_out/merge_final_23_LKL_1_2_truncada_reescalado.tif"
path_to_csv = r"/home/nico/dev/cicatrizes_certo/desenhadas/pontos_other_23_LKL_1_2.txt"

handSI = handlingSI(path_to_SAR_VV)
buildDS = buildDataset(r"/home/nico/dev/cicatrizes_certo/dataset", handSI.SARimage, handSI.raster, kernel = 5, points = None) #kernel = 5x5 (25 pixels)x10 bandas
buildDS.get_points(path_to_csv) 

## Codigo para treinar o modelo de deteccao de cicatrizes

In [1]:
#treinamento da rede neural
'''
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
'''

from Training.training import training
from time import time


start_time = time()

dataset = r"/home/nico/dev/cicatrizes_certo/dataset" #Dataset 
model_dir = r"/home/nico/dev/cicatrizes_certo/t_final/model/modeloLeneet10x10_kernel5x5.h5" #Dir para salvar o modelo

lbldic = {    	
		"cicatriz":3,
		"cicatriz2":1,
		"cultivo":2,
		"other":0,	

    }


trainer = training( #parametros para treinamento
	path_to_dataset=dataset, #dir do dataset
	save_to_dir_model=model_dir,  #dir para salvar o modelo
	lbl=lbldic,  #dicionario de classes
	bs= 1, # 32  #batch size
	epochs=200, # 200 #numero de epocas
	lr=0.00001,  #learning rate
	seed=60,  #seed
	numclasses=len(lbldic),  #numero de classes
	sizeofimage=5,  #tamanho da imagem
	splitDataset=0.2 #split do dataset
)

trainer.train() #treinamento

elapsed_time = time() - start_time
print(F"\nTempo de execucao: {elapsed_time}\n")

2023-11-29 22:29:16.370039: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 22:29:16.394875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 22:29:16.394892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 22:29:16.395507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 22:29:16.400082: I tensorflow/core/platform/cpu_feature_guar

4
Epoch 1/200


/home/nico/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (8294, 5, 5, 10) (10 channels).
  warnings.warn(
2023-11-29 22:29:20.028555: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-29 22:29:20.128347: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-11-29 22:29:20.848344: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fbdaef66b90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-29 22:29:20.848387: I external/local_xla/xla/service/service.cc:176]   Stream

8288/8294 [============================>.] - ETA: 0s - loss: 0.3356 - accuracy: 0.8790
Epoch 1: val_accuracy improved from -inf to 0.98457, saving model to /home/nico/dev/cicatrizes_certo/treinamento
INFO:tensorflow:Assets written to: /home/nico/dev/cicatrizes_certo/treinamento/assets


INFO:tensorflow:Assets written to: /home/nico/dev/cicatrizes_certo/treinamento/assets


...

INFO:tensorflow:Assets written to: /home/nico/dev/cicatrizes_certo/treinamento/assets


8294/8294 [==============================] - 52s 6ms/step - loss: 0.0177 - accuracy: 0.9952 - val_loss: 0.0081 - val_accuracy: 0.9981
Epoch 18/200
8288/8294 [============================>.] - ETA: 0s - loss: 0.0193 - accuracy: 0.9932
Epoch 18: val_accuracy did not improve from 0.99807
8294/8294 [==============================] - 51s 6ms/step - loss: 0.0193 - accuracy: 0.9932 - val_loss: 0.0087 - val_accuracy: 0.9961
Epoch 19/200
8286/8294 [============================>.] - ETA: 0s - loss: 0.0168 - accuracy: 0.9948
Epoch 19: val_accuracy did not improve from 0.99807
8294/8294 [==============================] - 49s 6ms/step - loss: 0.0168 - accuracy: 0.9948 - val_loss: 0.0071 - val_accuracy: 0.9976
Epoch 20/200
8289/8294 [============================>.] - ETA: 0s - loss: 0.0156 - accuracy: 0.9943
Epoch 20: val_accuracy did not improve from 0.99807
8294/8294 [==============================] - 50s 6ms/step - loss: 0.0156 - accuracy: 0.9943 - val_loss: 0.0072 - val_accuracy: 0.9966
Epoch 21

INFO:tensorflow:Assets written to: /home/nico/dev/cicatrizes_certo/treinamento/assets


8294/8294 [==============================] - 52s 6ms/step - loss: 0.0524 - accuracy: 0.9834 - val_loss: 0.0078 - val_accuracy: 0.9990
Epoch 23/200
8294/8294 [==============================] - ETA: 0s - loss: 0.0148 - accuracy: 0.9960
Epoch 23: val_accuracy did not improve from 0.99904
8294/8294 [==============================] - 50s 6ms/step - loss: 0.0148 - accuracy: 0.9960 - val_loss: 0.0077 - val_accuracy: 0.9971
Epoch 24/200
8288/8294 [============================>.] - ETA: 0s - loss: 0.0268 - accuracy: 0.9925
Epoch 24: val_accuracy did not improve from 0.99904
8294/8294 [==============================] - 50s 6ms/step - loss: 0.0268 - accuracy: 0.9925 - val_loss: 0.0071 - val_accuracy: 0.9976
Epoch 25/200
8292/8294 [============================>.] - ETA: 0s - loss: 0.0245 - accuracy: 0.9934
Epoch 25: val_accuracy did not improve from 0.99904
8294/8294 [==============================] - 50s 6ms/step - loss: 0.0245 - accuracy: 0.9934 - val_loss: 0.0077 - val_accuracy: 0.9961
Epoch 26

KeyboardInterrupt: 

## Confusion Matrix
## [[804  21   0]
## [  1 628   0]
## [  8   0 237]]


## Inferencia em imagens maiores classificando com base no modelo treinado e tamanho de kernel 5x5, resuiltando em uma imagem de mesmo tamanho da entrada com os pixel substituidos por cores correspondentes a classe

In [3]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2 as cv
from Visualization.visual import visual
from HandlingSI.handlingSI import handlingSI
from HandlingSI.utils import utils
from HandlingSI.handlingSI import handlingSI
import argparse
import cv2 as cv
from osgeo import gdal
import time
from time import time as tm
np.seterr(divide='ignore', invalid='ignore')
   
start_time = tm()
class inference(object):
	def __init__(self, image, rasterfromgdal, path_to_model, path_to_result, NoDataValue, kernel, lbl, colors):
		self.image = image
		self.rasterfromgdal = rasterfromgdal
		self.path_to_model = path_to_model
		self.path_to_result = path_to_result
		self.kernel = kernel
		self.result_image = None
		self.NoDataValue = NoDataValue
		self.obutil = utils()
		self.obhand = handlingSI()
		self.obhand.set_raster(self.rasterfromgdal)
		self.lbl = lbl
		self.colors = colors


	def test_model(self):
		
		CanaisImg=None
		bandasIm = []
		raster = self.rasterfromgdal
		print(f"Número de bandas: {raster.RasterCount} \n")
		if raster.RasterCount == 1:
			CanaisImg == 1
			self.image = raster.ReadAsArray()
			height = self.image.shape[0]
			width = self.image.shape[1]
			total_px = height*width 
			self.result_image = np.zeros((height,width), np.float32) #cria uma imagem de zeros para salvar o resultado
		else:
			for i in range(raster.RasterCount):								
				bandasIm.append(raster.GetRasterBand(i+1).ReadAsArray())	#salva as bandas em uma lista
			CanaisImg = raster.RasterCount #numero de canais da imagem
			image = cv.merge(bandasIm) #cria uma imagem com as bandas

			print(f"Before {image.shape}")
			K = self.kernel
			top = bottom = np.int64((K-1)/2) 
			print(f"top= bottom = {top}")
			left = right = np.int64((K-1)/2)
			print(f"left= right = {left}") 
			image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_REFLECT,) #reflete a borda da imagem
			self.obhand.saveSI(image= image,outFileName= r"/home/nico/dev/cicatrizes_certo/classificado/classificado9.tif",NoDataValue = self.NoDataValue) #salva a imagem com borda refletida
			print(f"After {image.shape}")

			height = image.shape[0] 
			width =  image.shape[1]  
			depth = CanaisImg #profundidade da imagem
			total_px = height*width*depth #calcula o total de pixels
			self.result_image = np.zeros((height,width,depth), np.float32) #cria uma imagem de zeros para salvar o resultado
			print(f"Total de pixels para processar: {total_px} \n")
		print(f"Profundidade: {image.shape[2]} \n") 
		
		model = load_model(self.path_to_model, compile=False) #carrega o modelo
		delta_kernel=int(np.trunc(self.kernel/2)) #calcula o delta do kernel
		self.result_image = np.zeros((height,width), np.float32) #cria uma imagem de zeros para salvar o resultado
		cont=0	
		
		self.obutil.printProgressBar(0, total_px, prefix = 'Progress:', suffix = 'Complete', length = 50) #barra de progresso
		for i in range(delta_kernel,height-delta_kernel): #loop para percorrer a imagem
			for j in range(delta_kernel,width-delta_kernel): #loop para percorrer a imagem
				if image.item(i,j,1) == self.NoDataValue: #se o pixel for igual ao valor de NoData
					self.result_image.itemset((i,j),self.NoDataValue)		 #salva o valor de NoData			
				else: #se nao for igual ao valor de NoData	
					refPt = (j, i) #pega as coordenadas do pixel
					sample=self.obutil.set_ROI(image,refPt,delta_kernel) #cria a ROI
					sample = img_to_array(sample) #converte a ROI para array
					#print(f"\n La forma de sample es: {sample.shape} \n")
					sample = np.expand_dims(sample, axis= 0) #expande as dimensoes da ROI
					#print(f"\n La forma de np.expand_dims es: {sample.shape} \n")
					prediction = model.predict(sample)[0] #faz a predicao
					#print(f"\n Prediction:  {prediction.shape} \n")
					value_predicted = self.colors[np.argmax(prediction)] #pega o valor da predicao					

					#print("value_predicted:",value_predicted)
					#if raster.RasterCount == 1:
					self.result_image.itemset((i,j),value_predicted)	 #salva o valor da predicao				
				
				self.obutil.printProgressBar(cont, total_px, prefix = 'Progress:', suffix = 'Complete', length = 50) #barra de progresso
				cont = cont + 1 #contador
				#print(f"Contador: {cont} \n")

		self.obhand.saveSI_inference(image=self.result_image,outFileName=self.path_to_result,NoDataValue= self.NoDataValue) #salva a imagem com o resultado 
   
path_to_SAR_VV = r"/home/nico/dev/cicatrizes_certo/classificado/classificado6.tif"
model_dir = r"/home/nico/dev/cicatrizes_certo/t_final/model/modeloEficient10x10_kernel5x5_62%.h5" 
name_of_classification = r"/home/nico/dev/cicatrizes_certo/t_final/classificado/inference_eficient_62%.tif" 

handSI = handlingSI(path_to_SAR_VV)	 #carrega a imagem

lbldic = { #dicionario de classes
		"other":0,
		"cicatriz":1,
		"cicatriz2":2,
		"cultivo":3,
	}

#nameclasses = [3,4,6,7,8,9,10,11,13,14]
nameclasses = [3,4,6,7] #Cores das classes para visualizacao da nova imagem

tester = inference(image=handSI.SARimage,  #parametros para inferencia
	rasterfromgdal=handSI.raster, #imagem  
	path_to_model=model_dir,  #modelo
	path_to_result=name_of_classification,  #resultado
	NoDataValue=-32768,  #valor de NoData
	kernel=5,  #kernel
	lbl=lbldic, #dicionario de classes
	colors=nameclasses) #cores das classes
tester.test_model() #inferencia

Número de bandas: 10 

Before (260, 189, 10)
top= bottom = 2
left= right = 2
bandera:  10
planos:  10
Bandas: 10; image: (264, 193, 10)
Bandas en saveSI (264, 193)
After (264, 193, 10)
Total de pixels para processar: 509520 

Profundidade: 10 

1/1 [==============================] - 0s 13ms/step----------| 0.006155% Comple


KeyboardInterrupt: 

## RESNET50-CLASSIFICATION-TEST-RESULT
![alt text](/home/nico/dev/cicatrizes_certo/t_final/training_results.png "RESNET50-CLASSIFICATION-TEST-RESULT")
## DENSENET-CLASSIFICATION-TEST-RESULT
![alt text](/home/nico/dev/cicatrizes_certo/t_final/training_results_99%.png "DENSENET-CLASSIFICATION-TEST-RESULT")
## KMEANS-CLASSIFICATION-TEST-RESULT
![alt text](/home/nico/dev/cicatrizes_certo/t_final/kmeans.png "KMEANS-CLASSIFICATION-TEST-RESULT")
## DENSENET-CLASSIFICATION-TEST-RESULT
![alt text](/home/nico/dev/cicatrizes_certo/t_final/classificado/classificado2.tif "DENSENET-CLASSIFICATION-TEST-RESULT")
